## Time 

In [ ]:
import datetime
 
dt_str = "2025-04-06T11:30:00Z"
dt = datetime.datetime.fromisoformat(dt_str)
 
print(dt.timestamp())  # вывод: 2009-05-28 16:15:00

## Best matching

In [3]:
# type: ignore 
import openpyxl
wb = openpyxl.load_workbook('examples/sunday_top_leagues.xlsx')
ws = wb.active

bets_api_fixutres: list[tuple[str, str, str]] = []

for row in ws.iter_rows(min_row=2, min_col=2, values_only=True):

    leagueV3 = row[0]
    home   = row[1]
    away   = row[2]
    bets_api_fixutres.append((leagueV3, home, away))

In [1]:
import ps3838api.api as ps
import ps3838api.tank as tank
event_matcher = tank.EventMatcher()

In [4]:
from ps3838api.models.event import Failure
from ps3838api.tank import EventInfo


matched_bets_api_events: list[Failure | EventInfo] = []
for leagueV3, home, away in bets_api_fixutres:
    matched_bets_api_events.append(
        event_matcher.get_league_id_and_event_id(leagueV3, home, away, force_local=True))

In [ ]:
from ps3838api.logic import find_event_by_id
from rapidfuzz import fuzz

result: list[tuple[float, str]] = []

for event, (leagueV3, home, away) in zip(matched_bets_api_events, bets_api_fixutres):
    if not isinstance(event, dict):
        print('🚫', leagueV3, home, '-', away, event)
        continue
    fixtureV3 = find_event_by_id(event_matcher.fixtures.data, event)
    assert fixtureV3 is not None
    score_home = fuzz.token_set_ratio(home, fixtureV3.get("home", ""))
    score_away = fuzz.token_set_ratio(away, fixtureV3.get("away", ""))
    total_score = score_home + score_away
    result.append((
        total_score, 
        f'{home} -> {fixtureV3.get("home", "")} | {away} -> {fixtureV3.get("away", "")}'
        )
    )

result.sort(key=lambda x: x[0])
for line in result:
    if line[0] // 2 > 75:
        continue
    print(line[0] // 2, '%', line[1])

## Tank Playground

In [1]:
from ps3838api.tank import EventMatcher, EventInfo


event_matcher = EventMatcher(league_ids=None)


In [4]:
league = 'Spain La Liga'
home = 'Alaves'
away = 'Real Madrid'

In [ ]:
event_matcher.get_league_id_and_event_id(league, home, away)

In [ ]:
from ps3838api.matching import magic_find_event


event = magic_find_event(event_matcher.fixtures.data, league, home, away)

assert isinstance(event, dict)

event


In [50]:
event_matcher.odds.is_live = True

In [ ]:
assert isinstance(event, dict)

odds = event_matcher.get_odds(event)
odds

In [ ]:
from ps3838api.logic import filter_odds


odds_response_full = ps.get_odds(is_live=True)

filter_odds(odds_response_full, event_id=event['eventId'])

In [ ]:
odds_response_full

In [18]:
with open('event_matcher.odds.data.json', 'w') as file:
    json.dump(event_matcher.odds.data, file, indent=4)

In [ ]:
import json

with open('temp/responses/odds_snapshot_1744036330.json') as file:
    odds_snapshot = json.load(file)

with open('temp/responses/odds_delta_1744207459.json') as file:
    odds_delta = json.load(file)

from ps3838api.logic import merge_odds_response
from ps3838api.logic import filter_odds


filter_odds(merge_odds_response(odds_snapshot, odds_delta), event_id=event['eventId'])


In [36]:
assert isinstance(event, dict)

odds_event = filter_odds(odds_snapshot, event_id=event['eventId'])

In [ ]:
from ps3838api.totals import get_best_total_line

assert isinstance(odds, dict)

total_line = get_best_total_line(odds)
total_line

In [48]:
import ps3838api.api as ps

stake_usdt = 1.0
assert isinstance(event, dict)


place_bet_response = ps.place_straigh_bet(
    stake=stake_usdt,
    event_id=event['eventId'],
    bet_type='TOTAL_POINTS',
    line_id=total_line.get('lineId', None),
    alt_line_id=total_line.get('altLineId', None),
    side='OVER',
    handicap=total_line['points']
    
)
place_bet_response

## Live bets

In [7]:
live_fixtures = ps.get_fixtures(is_live=True)

In [8]:
league = 'England - Premier League'
home = 'Brentford'

for leagueV3 in live_fixtures['league']:
    for fixture in leagueV3['events']:
        event: EventInfo = {'eventId': fixture['id'], 'leagueId': leagueV3['id']}
        if leagueV3['name'] == league and fixture['home'] == home:
            print(leagueV3['name'], fixture['home'], fixture['away'], sep=' | ')
            break  # Break the inner loop...
    else:
        continue  # Continue if the inner loop wasn't broken.
    break  # Inner loop was broken, break the outer.


In [ ]:
# odds = event_matcher.get_odds(event)
# print(type(odds))

In [25]:
from ps3838api.models.odds import OddsResponse


odds: OddsResponse = {} # type: ignore

In [ ]:
from ps3838api.totals import get_best_total_line

odds = ps.get_odds(event_ids=[event['eventId'], ], since=odds.get('last', None))

total_line = get_best_total_line(odds['leagues'][0]['events'][0])
assert total_line is not None
total_line

In [34]:
stake_usdt = 1.0

In [ ]:
assert total_line is not None

place_bet_response = ps.place_straigh_bet(
    stake=stake_usdt,
    event_id=event['eventId'],
    bet_type='TOTAL_POINTS',
    line_id=total_line.get('lineId', None),
    alt_line_id=total_line.get('altLineId', None),
    side='OVER',
    handicap=total_line['points']
) 
place_bet_response

In [7]:
bets = ps.get_bets(bet_ids=[3273866035, ])

In [ ]:
bet = bets['straightBets'][0]
bet